## Project: Analyse data to identify mobile apps which tend to have more users.

I'm a Data Anayst for a company which builds mobile apps that are free to download on Google play store and Apple app store. The main source of revenue for our apps is by in-app ads. This means revenue is mostly influenced by number of users who use our apps — the more users that see and engage with the ads, the better for the companys revenue. 

The objective as a Data Analyst for this project is to analyze data to help the company understand type of apps likely to attract more users. This has the potential to help the company significantly improve our mobile app user base and subsequently increase the companys mobile revenue stream.

Based on published material available on the web. The below datasets will be used during our data analysis.

- Please click [here](https://www.kaggle.com/lava18/google-play-store-apps) to access documentation on google play store dataset 

- Please click [here](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps) to access documentation on app store dataset 

Begin analysis by first reading in the sample data downloaded from the internet. The data is then stored in separate lists for Google apps and Apple apps.  

In [24]:
# Read the data
import csv
file_google = open('googleplaystore.csv') 
read_google = csv.reader(file_google)
data_google = list(read_google)
data_google_header = data_google[0]
data_google = data_google[1:]

file_apple = open('AppleStore.csv')
read_apple = csv.reader(file_apple)
data_apple = list(read_apple)
data_apple_header = data_apple[0]
data_apple = data_apple[1:]

The defined function `data_explore` will help with sampling data from the google and apple datasets

In [25]:
# This function prints rows from the dataset
def data_explore(data, start, end, rows_and_columns=False):
    data_slice = data[start:end]
    for row in data_slice:
        print(row)
        print('\n')
        
    if rows_and_columns:
        print('Number of rows:', len(data))
        print('Number of columns:', len(data[0]))

Based on the sample data and headers displayed below, the following data points which have details about the app name, category, reviews, downloads, and price will be very useful for getting more data insights on which category / genre of published apps tend to do well on these app stores, furthermore, this will be analysed by considering price, the company is more interested in data relating to free mobile apps in the playstore and app store.

In [26]:
# Preview the data
print(data_google_header)
data_explore(data_google, 1, 6, True)

print ('\n')

print(data_apple_header)
data_explore(data_apple, 1, 6, True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up']


['Paper flowers instructions', 'ART_AND_DESIGN', '4.4', '167', '5.6M', '50,000+', 'Free', '0', 'Everyone', 'Art & D

### Data cleansing 

Before we progress further ahead, we need to clean the
datasets (data cleaning) by detecting and removing:
- duplicate data
- inaccurate data
- non-english apps like 爱奇艺PPS -《欢乐颂2》电视剧热播.
- paid apps  

For the time being, my company is only interested in developing
`free` mobile appss for an english speaking audience hence we have to
streamline our dataset for these reasons.

In [27]:
# Based on published information on the discussion 
# section of the google dataset, the row index [10472]
# (excluding header) is missing its category datapoint.
print (data_google[10472])
del data_google[10472]

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


The defined function `duplicate` will help with finding duplicate data from the google and apple datasets

In [28]:
#This function checks for duplicates 
def duplicate(data,index,dataset):
    duplicate_list = []
    unique_list = []
    for app in data:
        name=app[index]
        if name in unique_list:
            duplicate_list.append(name)
        else:
            unique_list.append(name)
    print ('There are' , len(duplicate_list) , 'apps in the ' + dataset)
    print ('\n')
    print ('This is a duplicate app in the ' + dataset , ':', duplicate_list[:16])
    print ('\n')

In [29]:
duplicate(data_google,0,'playstore dataset')
duplicate(data_apple,1,'appstore dataset')

There are 1181 apps in the playstore dataset


This is a duplicate app in the playstore dataset : ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack', 'FreshBooks Classic', 'Insightly CRM', 'QuickBooks Accounting: Invoicing & Expenses', 'HipChat - Chat Built for Teams', 'Xero Accounting Software', 'MailChimp - Email, Marketing Automation']


There are 2 apps in the appstore dataset


This is a duplicate app in the appstore dataset : ['Mannequin Challenge', 'VR Roller Coaster']




The higher the number of reviews, the more recent the data should be, hence for the data analysis, only the row with the highest number of reviews will be kept while other duplicate entries will be removed.

In [30]:
reviews_max={}
for app in data_google:
    name=app[0]
    n_reviews=float(app[3])
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
    elif name not in reviews_max:
        reviews_max[name] = n_reviews
print ('There will be', len(reviews_max), 'rows left when duplicates are removed from the playstore dataset')

There will be 9659 rows left when duplicates are removed from the playstore dataset


The `data_google_clean` list will not have duplicate enteries because the duplicates have been removed by keeping only data rows with the highest review.

In [31]:
data_google_clean=[]
already_added=[]
for app in data_google:
    name=app[0]
    n_reviews=float(app[3])
    if n_reviews == reviews_max[name] and name not in already_added:
        data_google_clean.append(app)
        already_added.append(name)
data_explore(data_google_clean, 1, 6, True)

['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up']


['Paper flowers instructions', 'ART_AND_DESIGN', '4.4', '167', '5.6M', '50,000+', 'Free', '0', 'Everyone', 'Art & Design', 'March 26, 2017', '1.0', '2.3 and up']


['Smoke Effect Photo Maker - Smoke Editor', 'ART_AND_DESIGN', '3.8', '178', '19M', '50,000+', 'Free', '0', 'Everyone', 'Art & Design', 'April 26, 2018', '1.1', '4.0.3 and up']


Number of rows: 9659
Number of columns: 13


There are some mobile apps in the datasets which are for non-English audience. These will be removed since the company is more focused on developing English based apps. The `ord` built in python function will be used to reduce the data set to only the apps which have the character range between `0-127` in ASCII [American Standard Code for Information Interchange].

In [32]:
# Preview data on non-english apps
print(data_apple[813])
print(data_apple[6731])

['445375097', '爱奇艺PPS -《欢乐颂2》电视剧热播', '224617472', 'USD', '0.0', '14844', '0', '4.0', '0.0', '6.3.3', '17+', 'Entertainment', '38', '5', '3', '1']
['1120021683', '【脱出ゲーム】絶対に最後までプレイしないで 〜謎解き＆ブロックパズル〜', '77551616', 'USD', '0.0', '0', '0', '0.0', '0.0', '1.3', '12+', 'Games', '38', '0', '1', '1']


The `data_english` function will help to remove data on non-English related mobile apps.

In [33]:
# This function removes rows on non-English mobile apps
def data_english(dstr):
    count=0
    for char in dstr:
        dord=ord(char)
        #print(dord)
        if dord > 127:
            count+=1
    if count > 3:
        return False
    else:
        return True
print(data_english('Instagram'))
print(data_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(data_english('Docs To Go™ Free Office Suite'))
print(data_english('Instachat 😜'))

True
False
True
True


The `data_eng_clean` and `data_eng_apple` list will only contain data on English related mobile apps because the non-English related rows have been removed from the datasets.

In [34]:
data_eng_google=[]
data_eng_apple=[]
for app in data_google_clean:
    name=app[0]
    if data_english(name):
        data_eng_google.append(app)

for app in data_apple:
    name=app[1]
    if data_english(name):
        data_eng_apple.append(app)

In [35]:
# Preview data
data_explore(data_eng_google, 0, 3, True)
data_explore(data_eng_apple, 0, 3, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 9614
Number of columns: 13
['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'G

The paid apps will be removed from the datasets by creating a final data list which does not have mobile apps with a price that is greater than `0.0`. The `data_final_google` and `data_final_apple` data list will be used for recommendation to the company when the data points have been adequately analysed further.

In [36]:
data_final_google=[]
data_final_apple=[]

for app in data_eng_google:
    price=app[7]
    if price == '0':
        data_final_google.append(app)

for app in data_eng_apple:
    price=app[4]
    if price == '0.0':
        data_final_apple.append(app)

print('There are' , len(data_final_google) , 'mobile apps remaining in the playstore dataset')
print('There are' , len(data_final_apple) , 'mobile apps remaining in the appstore dataset')

There are 8864 mobile apps remaining in the playstore dataset
There are 3222 mobile apps remaining in the appstore dataset


### Prepare the data for final analysis by building computations on percentages
The data is ready to be analysed. The objective as a Data Analyst for this project was to analyze the sample data on mobile apps in the Google playstore and Apple apps store to help the company understand type of apps likely to attract more users. This has the potential to help the company significantly improve its mobile app user base and subsequently increase the companys mobile revenue stream.

In [37]:
# This function builds a frequency table for the 
# prime_genre data column of the app store dataset 
# and the genres and catrgory column of the playstore 
# dataset which will be displayed in percentages
def data_freq(data, index):
    datafreq={}
    total=0
    for app in data:
        total +=1
        datac = app[index]
        if datac in datafreq:
            datafreq[datac]+=1
        else:
            datafreq[datac]=1
    
    data_perc={}
    for app in datafreq:
        perc = (datafreq[app]/total)*100
        data_perc[app] = perc
    return data_perc

# This function will display the percentages 
# developed in data_freq by descending order
def display_table(data, index):
    dataf = data_freq(data, index)
    data_display = []
    for key in dataf:
        key_val_as_tuple = (dataf[key], key)
        data_display.append(key_val_as_tuple)
        
    data_sorted = sorted(data_display, reverse = True)
    for entry in data_sorted:
        print(entry[1], ':', entry[0])

Based on the data below generated from the Apple app store dataset, the `Games` genre seems to be miles ahead in representation with 58% while `Entertainment` genre takes distant second place with 7.8%. The genral impression is that mobile apps designed for entertainment such as games, social networking seem to be more available than mobile apps designed for practical purposes for instance (productivity, news, weather, finace).

In [38]:
display_table(data_final_apple, -5)

Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


Based on the data below generated from the Google playstore dataset, the `Family` and `Game` genre seem to be have a higher availability percentage with a combined percentage of 28.64% taking first and second place respectively. The genral impression here as was similarly observed in the app store dataset is that mobile apps designed for entertainment seem to be highly represented than mobile apps designed for practical purposes for instance (productivity, finance, medical).

But, based on further observation, unlike the app store dataset which had a combined percentage of 66% for the top two popular genres geared towards fun/entertainment, the google play store only has 28.64% in direct comparison. This gives the impression that productivity apps tend to also do well in the google playstore dataset. For instance mobile apps in the genres: tools, business, productivety, finance and medical combine for a percentage 24.11%. 

This data shows a more balanced landscape of both practical and fun apps in the Google playstore compared with the Apple app store which seems to be highly dominated by mobile apps designed for fun / entertainment.

In [39]:
display_table(data_final_google, 1)

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 

The below will help to discover the most popular genres in the app store dataset, based on the total number of users, remember, the objective is to find the genre of apps which have the most users for both datasets. For the playstore dataset, the installs column will be useful, but for the app store dataset, the total number of user ratings will be used as a proxy, which we can find in the rating_count_tot column.

In [40]:
data_genres_apple = data_freq(data_final_apple, -5)

for genre in data_genres_apple:
    total = 0
    len_genre = 0
    for app in data_final_apple:
        data_genre = app[-5]
        if data_genre == genre:            
            ratings = float(app[5])
            total += ratings
            len_genre += 1
    ratings_avg = total / len_genre
    print(genre, ':', ratings_avg)

Games : 22788.6696905016
Education : 7003.983050847458
Productivity : 21028.410714285714
Food & Drink : 33333.92307692308
Photo & Video : 28441.54375
Catalogs : 4004.0
Lifestyle : 16485.764705882353
Health & Fitness : 23298.015384615384
Book : 39758.5
Utilities : 18684.456790123455
Music : 57326.530303030304
Social Networking : 71548.34905660378
Shopping : 26919.690476190477
News : 21248.023255813954
Reference : 74942.11111111111
Weather : 52279.892857142855
Finance : 31467.944444444445
Travel : 28243.8
Sports : 23008.898550724636
Entertainment : 14029.830708661417
Navigation : 86090.33333333333
Medical : 612.0
Business : 7491.117647058823


The data shows that the highest number of user reviews belong to `Navigation` genre, but this figure seems to be highly influenced by the popularity of google maps and waze, which have close to half a million user reviews together. This pattern seems to be the same with the `Social Networking` genre where the average number of user reviews is highly influenced by popular organisations for instance Facebook and Microsoft. 

In [41]:
for app in data_final_apple:
    if app[-5] == 'Navigation':
        print(app[1], ':', app[5])

Waze - GPS Navigation, Maps & Real-time Traffic : 345046
Google Maps - Navigation & Transit : 154911
Geocaching® : 12811
CoPilot GPS – Car Navigation & Offline Maps : 3582
ImmobilienScout24: Real Estate Search in Germany : 187
Railway Route Search : 5


Besides the `Navigation` and `Social Networking` genre which have data that might be skewing the results due to the popularity of certain high profile mobile apps, the `Reference` genre seems to be doing quite well on the app store with 74,942 user based reviews on the average. By diving further into this genre, below, it could be observered that the Bible and Dictionary.com mobile apps which skew up the average rating for the `Reference` genre. 

Publishing a free mobile app in the `Reference` genre could be promising for the company since the objective is to significantly increase the user base, this genre seems to be doing very well and keeping up with the popularity of genres like navigation and social networking. Publishing mobile versions of highly popular books could prove to be a success for bringing in more mobile users. Possibly, this could be a book which could be geared towards family / fun by integrating puzzles, mini games, thereby building parts of these highly popular genres into the possible ideas for the potential mobile apps. 

In [42]:
for app in data_final_apple:
    if app[-5] == 'Reference':
        print(app[1], ':', app[5])

Bible : 985920
Dictionary.com Dictionary & Thesaurus : 200047
Dictionary.com Dictionary & Thesaurus for iPad : 54175
Google Translate : 26786
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran : 18418
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition : 17588
Merriam-Webster Dictionary : 16849
Night Sky : 12122
City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE) : 8535
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools : 4693
GUNS MODS for Minecraft PC Edition - Mods Tools : 1497
Guides for Pokémon GO - Pokemon GO News and Cheats : 826
WWDC : 762
Horror Maps for Minecraft PE - Download The Scariest Maps for Minecraft Pocket Edition (MCPE) Free : 718
VPN Express : 14
Real Bike Traffic Rider Virtual Reality Glasses : 8
教えて!goo : 0
Jishokun-Japanese English Dictionary & Translator : 0


The below displays the total count on installs for data in the playstore dataset but this data point is not precise, for instance, we are not particularly certain whether an app with 100,000+ installs has 100,000 installs, 200,000, or 350,000.

In [43]:
# Preview data based on installs 
display_table(data_final_google, 5)

1,000,000+ : 15.726534296028879
100,000+ : 11.552346570397113
10,000,000+ : 10.548285198555957
10,000+ : 10.198555956678701
1,000+ : 8.393501805054152
100+ : 6.915613718411552
5,000,000+ : 6.825361010830325
500,000+ : 5.561823104693141
50,000+ : 4.7721119133574
5,000+ : 4.512635379061372
10+ : 3.5424187725631766
500+ : 3.2490974729241873
50,000,000+ : 2.3014440433213
100,000,000+ : 2.1322202166064983
50+ : 1.917870036101083
5+ : 0.78971119133574
1+ : 0.5076714801444043
500,000,000+ : 0.2707581227436823
1,000,000,000+ : 0.22563176895306858
0+ : 0.04512635379061372
0 : 0.01128158844765343


The below helps to compute the average number of installs for each genre (category) in the google playstore dataset for a better data analysis and subsequently mobile app recommendation.

In [44]:
google_categories = data_freq(data_final_google, 1)

for category in google_categories:
    total = 0
    len_category = 0
    for app in data_final_google:
        category_app = app[1]
        if category_app == category:            
            n_installs = app[5]
            n_installs = n_installs.replace(',', '')
            n_installs = n_installs.replace('+', '')
            total += float(n_installs)
            len_category += 1
    avg_n_installs = total / len_category
    print(category, ':', avg_n_installs)

PARENTING : 542603.6206896552
FAMILY : 3695641.8198090694
BEAUTY : 513151.88679245283
NEWS_AND_MAGAZINES : 9549178.467741935
ART_AND_DESIGN : 1986335.0877192982
MEDICAL : 120550.61980830671
AUTO_AND_VEHICLES : 647317.8170731707
GAME : 15588015.603248259
PRODUCTIVITY : 16787331.344927534
WEATHER : 5074486.197183099
DATING : 854028.8303030303
SOCIAL : 23253652.127118643
SHOPPING : 7036877.311557789
HEALTH_AND_FITNESS : 4188821.9853479853
PERSONALIZATION : 5201482.6122448975
HOUSE_AND_HOME : 1331540.5616438356
COMICS : 817657.2727272727
LIFESTYLE : 1437816.2687861272
FOOD_AND_DRINK : 1924897.7363636363
ENTERTAINMENT : 11640705.88235294
FINANCE : 1387692.475609756
LIBRARIES_AND_DEMO : 638503.734939759
SPORTS : 3638640.1428571427
BOOKS_AND_REFERENCE : 8767811.894736841
PHOTOGRAPHY : 17840110.40229885
EDUCATION : 1833495.145631068
EVENTS : 253542.22222222222
TOOLS : 10801391.298666667
MAPS_AND_NAVIGATION : 4056941.7741935486
TRAVEL_AND_LOCAL : 13984077.710144928
COMMUNICATION : 38456119.1672

The google playstore seems to be dominated by highly popular apps from high profile organisations. The data shows that communication apps have the most installs: 38,456,119 but like previously observed in the app store dataset, this number is skewed by highly popular social networking apps which tend to have more than one billion installs for instance whatsapp, facebook messenger, youtube, and instagram. 

In [45]:
for app in data_final_google:
    if app[1] == 'COMMUNICATION' and (app[5] == '1,000,000,000+'
                                      or app[5] == '500,000,000+'
                                      or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

WhatsApp Messenger : 1,000,000,000+
imo beta free calls and text : 100,000,000+
Android Messages : 100,000,000+
Google Duo - High Quality Video Calls : 500,000,000+
Messenger – Text and Video Chat for Free : 1,000,000,000+
imo free video calls and chat : 500,000,000+
Skype - free IM & video calls : 1,000,000,000+
Who : 100,000,000+
GO SMS Pro - Messenger, Free Themes, Emoji : 100,000,000+
LINE: Free Calls & Messages : 500,000,000+
Google Chrome: Fast & Secure : 1,000,000,000+
Firefox Browser fast & private : 100,000,000+
UC Browser - Fast Download Private & Secure : 500,000,000+
Gmail : 1,000,000,000+
Hangouts : 1,000,000,000+
Messenger Lite: Free Calls & Messages : 100,000,000+
Kik : 100,000,000+
KakaoTalk: Free Calls & Text : 100,000,000+
Opera Mini - fast web browser : 100,000,000+
Opera Browser: Fast and Secure : 100,000,000+
Telegram : 100,000,000+
Truecaller: Caller ID, SMS spam blocking & Dialer : 100,000,000+
UC Browser Mini -Tiny Fast Private & Secure : 100,000,000+
Viber Mess

Based on further analysis, the genres for `Game` and `Books and Reference` seem to do quite well with the average number of installs being 15,588,015 and 8,767,811 respectively. This data is insightful because the previous recommendation for the app store could potentially be used for the google playstore market as well meaning the company could practically launch the same mobile app that will be compatible on both markets.

In [46]:
for app in data_final_google:
    if app[1] == 'BOOKS_AND_REFERENCE':
        print(app[0], ':', app[5])

E-Book Read - Read Book for free : 50,000+
Download free book with green book : 100,000+
Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Free Panda Radio Music : 100,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
English Grammar Complete Handbook : 500,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
Google Play Books : 1,000,000,000+
AlReader -any text book reader : 5,000,000+
Offline English Dictionary : 100,000+
Offline: English to Tagalog Dictionary : 500,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
Recipes of Prophetic Medicine for free : 500,000+
ReadEra – free ebook reader : 1,000,000+
Anonymous caller detection : 10,000+
Ebook Reader : 5,000,000+
Litnet - E-books : 100,000+
Read books online : 5,000,000+
English to Urdu Dictionary : 500,000+
eBoox: book reader fb2 epub zip : 1,000,000+
English Persian Dictionary : 500,000+
Flybook : 500,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
E

### Recommendation

Publishing a free mobile app in the `Books and Reference` genre could be promising for the company since the objective is to significantly increase the user base, this genre seems to be doing very well and keeping up with the popularity of genres geared towards fun and entertainement like games, social networking, photography and video. This pattern is observed for both sample datasets from the google playstore market and apple app store. 

Publishing the mobile version of a highly popular book could prove to be a success for bringing in more mobile users. Possibly, this could be a book which could be geared towards both family and fun category by trying to integrate puzzles, mini games, quizzes, thereby building parts of the highly popular genres on the market into the same mobile app, possible ideas for the soon to be developed mobile apps from the company.